In [6]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 29 16:47:06 2019

@author: Antanas
"""

import math 
import argparse
import dendropy
import random
import numpy as np
#import simulation


import sampling
import beastxmlwriter
import phyrexxmlwriter
import treegenerator

import os

from dendropy.interop import seqgen

num_tips = 500
sample_size = 50
other_sample_size = 50
sigma = 1
alignment_length = 1000


tree = treegenerator.generate_yule_tree(num_tips)

#tree = simulation.simulate_brownian(tree, sigma, 2)

s = seqgen.SeqGen()

d0 = s.generate(tree)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Antanas\\AppData\\Local\\Temp\\tmpw08qppiu'

In [15]:
import dendropy
import random
import math 
from dendropy.simulate import treesim
import numpy as np
import treegenerator

tree=treegenerator.generate_yule_tree(5)
print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=False))


                                                                                                                /s0007
/---------------------------------------------------------------------------------------------------------------+     
|                                                                                                               \s0008
+                                                                                                                     
|                                         /--------------------------------------------------------------------- s0005
|                           /-------------+                                                                           
\---------------------------+             \--------------------------------------------------------------------- s0006
                            |                                                                                         
                            \-------------------

In [16]:
import dendropy
import random
import math 
from dendropy.simulate import treesim
import numpy as np


def generate_yule_tree(num_tips):
    lamb = 1
    names = []

    #lamb = 1
    
    #if there are N tips, there must be 2N-1 nodes
    for i in range(2*num_tips-1):
        if i < 10: 
            names.append("s000"+str(i))
        elif i < 100:
            names.append("s00"+str(i))
        elif i < 1000:
            names.append("s0"+str(i))
        else:
            names.append("s"+str(i))
            
    taxon_namespace = dendropy.TaxonNamespace(names)
    tree = dendropy.Tree(taxon_namespace=taxon_namespace)

    time = 0
    name_index = 0
    current_nodes = []
    tree.seed_node.taxon = taxon_namespace.get_taxon(names[name_index])
    name_index=name_index+1
    #tree.seed_node.age=0
    current_nodes.append(tree.seed_node)
    for i in range(num_tips-1):        
        time_to_split=random.expovariate(lamb*len(current_nodes))
        if i ==0:
            time=0
        else:
            time = time_to_split+time
        splitting_index = random.randint(0, len(current_nodes)-1)
        parent_node = current_nodes[splitting_index]

        node1 = dendropy.Node(taxon=taxon_namespace.get_taxon(names[name_index]))
        name_index = name_index+1
        node2 = dendropy.Node(taxon=taxon_namespace.get_taxon(names[name_index]))
        name_index = name_index+1

        parent_node.set_child_nodes([node1, node2])

        parent_node.time=time
#            node1.edge_length = time-parent_node.age
#            node2.edge_length = time-parent_node.age

        current_nodes.pop(splitting_index)
        current_nodes.append(node1)
        current_nodes.append(node2)
    for node in current_nodes:
        node.time = time

    for node in tree.preorder_node_iter():
        if node.time > 0:
            node.edge_length=node.time-node.parent_node.time    
    return tree

tree=generate_yule_tree(10)
tree.calc_node_ages()


print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=False))


                             /---------------------------------------------------------------------------------- s0003
/----------------------------+                                                                                        
|                            |                                                                        /--------- s0009
|                            \------------------------------------------------------------------------+               
|                                                                                                     \--------- s0010
|                                                                                                                     
|                                                                                                      /-------- s0011
+                                                                                             /--------+              
|                                               

In [19]:
tree = treesim.birth_death_tree(birth_rate=1, death_rate=0, num_total_tips=5)
tree.seed_node.edge_length = 0
print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=False))

                                                                   /----------------------------------------------- T5
                                                                   |                                                  
/------------------------------------------------------------------+                                               /T4
|                                                                  |                   /---------------------------+  
|                                                                  \-------------------+                           \T3
+                                                                                      |                              
|                                                                                      \--------------------------- T1
|                                                                                                                     
\-----------------------------------------------

In [1]:
import dendropy
import random
import math 
from dendropy.simulate import treesim
import numpy as np

# original list
s1 = "[&R] (A,(B,C)D)E;"
tree1 = dendropy.Tree.get(data=s1, schema="newick", suppress_internal_node_taxa=False)
print(tree1.as_ascii_plot(show_internal_node_labels=True))
print(tree1.taxon_namespace)
tree1.find_node_with_taxon_label("B").edge_length=7
tree1.find_node_with_taxon_label("C").edge_length=3
tree1.find_node_with_taxon_label("A").edge_length=4
node_D =tree1.find_node_with_taxon_label("D")
node_D.edge_length=2



print(tree1.as_ascii_plot(plot_metric='length', show_internal_node_labels=True))
tree2=tree1.extract_tree_without_taxa_labels(["C"])

print(tree2.as_ascii_plot(plot_metric='length', show_internal_node_labels=True))

/------------------------------------------------------------------------------------------------------------------- A
E                                                                                                                     
|                                                         /--------------------------------------------------------- B
\---------------------------------------------------------D                                                           
                                                          \--------------------------------------------------------- C
                                                                                                                      
                                                                                                                      
['A', 'B', 'C', 'D', 'E']
/-------------------------------------------------- A                                                                 
E                     

In [2]:


#this function generates borwnian motion for a given tree
def simulate_brownian(t, sigma, dimension):
    #t is the tree
    #sigma is the standard deviation of the brownian motion
    #dimension is the number of dimensions in which we generate the random walk
    for node in t.preorder_node_iter():
        if node.parent_node is None:
            node.X = float(0)
            node.displacementX = float(0)
            if dimension==2:
                node.Y = float (0)
                node.displacementY = float(0)
        else:
            node.displacementX = random.gauss(0, sigma*math.sqrt(node.edge.length))
            node.X = node.parent_node.X+node.displacementX            
            if dimension==2:
                node.displacementY = random.gauss(0, sigma*math.sqrt(node.edge.length))
                node.Y = node.parent_node.Y+node.displacementY            
    return t

#this function calculates the time from the seed node to the present (seed node has time 0)
def calculate_times(t):
    for node in t.preorder_node_iter():
        if node.parent_node is None:
            node.time = 0
        else:
            node.time = node.parent_node.time+node.edge.length
            
    return t 


#debug code bellow

def generatebd(br, dr, mt):
    t = treesim.birth_death_tree(birth_rate=br, death_rate=dr, max_time=mt, is_retain_extinct_tips=True, is_add_extinct_attr=True)
    
    index = 0
    namespace = [];
    
    for node in t.preorder_node_iter():
        index=index+1
        namespace.append("T"+str(index))
    
    #name all nodes instead of just leaves
    taxon_namespace = dendropy.TaxonNamespace(namespace)
    t.taxon_namespace=taxon_namespace
    index=0
    for node in t.preorder_node_iter():
        index=index+1
        node.taxon=t.taxon_namespace.get_taxon("T"+str(index))
    
    #distance to root
    t=calculate_times(t)        
    return t
          
    
tree = generatebd(1, 0.8, 3.0)
tree= simulate_brownian(tree, 1,2)

print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=True))

d=dendropy.model.discrete.hky85_chars(kappa=3, mutation_rate=0.1, seq_len=1000, tree_model=tree, retain_sequences_on_tree=False)
        
for node in tree.postorder_node_iter(): 
    if not hasattr(node, 'extinct_ancestor'):
        child_extinct = False
        for child in node.child_node_iter():
            if child.extinct_ancestor:
                child_extinct =True
        node.extinct_ancestor = child_extinct

print("\n is tip ancestor of an extinct node?:")
for node in tree.postorder_node_iter():    
    print("%s : %s" % (node.taxon.label, node.extinct_ancestor))


                                    /----------------- T3                                                             
                         /----------T2                                                                                
                         |          \------- T4                                                                       
                         |                                                                                            
                         |           /-------------- T7                                                               
                         |       /---T6                                                                               
                         |       |   |/-- T9                                                                          
                         |       |   \T8                                                                              
                         T1      |    |         

In [10]:
#drawing migration diagram

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


xcoords=[]
ycoords=[]

for node in tree.preorder_node_iter():
    xcoords.append(node.X)
    ycoords.append(node.Y)
    
tree.write(path="output.tre", schema="newick")
d.write(path="fastaoutput.tre", schema="fasta")

plt.scatter(xcoords, ycoords)
for node in tree.preorder_node_iter():
    if node.parent_node is not None:
        plt.arrow(node.parent_node.X,node.parent_node.Y, node.displacementX,node.displacementY)
plt.show()


AttributeError: 'Node' object has no attribute 'Y'

In [4]:
def generate_star_tree2():
    num_tips = 10
    branch_length = 1
    fake_step = 0.000000001
    names = [] 
    
    for i in range(num_tips):
        names.append("T"+str(i))   
        
    taxon_namespace = dendropy.TaxonNamespace(names)
    tree = dendropy.Tree(taxon_namespace=taxon_namespace)
#     fake_node = tree.seed_node
    
#     tree.seed_node.X = 0
#     tree.seed_node.time = 0
    
#     for i in range(num_tips):
#         if i ==num_tips-1:
#             node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
#             node.edge_length = branch_length
#             node.X = random.gauss(0,1)
#             node.time = branch_length+fake_step*i
#             fake_node.add_child(node)
#         else:
#             node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
#             node.edge_length = branch_length
#             node.X = random.gauss(0,1)
#             node.time = branch_length+fake_step*i
            
#             new_fake_node =dendropy.Node()
#             new_fake_node.edge_length = fake_step
#             new_fake_node.X = 0
#             new_fake_node.time = branch_length+fake_step*(i+1)
#             fake_node.add_child(new_fake_node)
#             fake_node = new_fake_node
#     return tree
    
    current_seed=dendropy.Node()
    current_seed.edge_length = fake_step
    i=0
    while i <num_tips:
        if i ==0:
            node1 = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
            node1.edge_length = branch_length
            node1.X = random.gauss(0,1)
            current_seed.add_child(node1)
            i=i+1
            
            node2 = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
            node2.edge_length = branch_length
            node2.X = random.gauss(0,1)
            current_seed.add_child(node2)
            current_seed.X=0
            i=i+1
        elif i == num_tips-1:
            node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
            node.edge_length = branch_length
            node.X = random.gauss(0,1)
            i=i+1
            tree.seed_node.X=0
            tree.seed_node.add_child(node)
            tree.seed_node.add_child(current_seed)
        else:
            current_seed2=dendropy.Node()
            current_seed2.edge_length = fake_step
            
            node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
            node.edge_length = branch_length
            node.X = random.gauss(0,1)
            i=i+1
            
            current_seed2.add_child(node)
            current_seed2.add_child(current_seed)
            current_seed= current_seed2
            current_seed.X=0
          
    return tree
tree = generate_star_tree2()
print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=False))

/----------------------------------------------------------------------------------------------------------------- T9 
+                                                                                                                     
|----------------------------------------------------------------------------------------------------------------- T8 
+                                                                                                                     
|----------------------------------------------------------------------------------------------------------------- T7 
+                                                                                                                     
|----------------------------------------------------------------------------------------------------------------- T6 
+                                                                                                                     
|-----------------------------------------------

In [5]:
def generate_birthdeath_tree(br, dr, num_extinct):
    t = treesim.birth_death_tree(birth_rate=br, death_rate=dr, num_extinct_tips=num_extinct, is_retain_extinct_tips=True, is_add_extinct_attr=True)
    #t.print_plot()    
    
    index = 0
    namespace = [];
    
    for node in t.preorder_node_iter():
        index=index+1
        namespace.append("T"+str(index))
    
    #name all nodes instead of just leaves
    taxon_namespace = dendropy.TaxonNamespace(namespace)
    t.taxon_namespace=taxon_namespace
    index=0
    for node in t.preorder_node_iter():
        index=index+1
        node.taxon=t.taxon_namespace.get_taxon("T"+str(index))
    
    t =prune_nodes(t)
    
    #distance to root
    t=calculate_times(t)
        
    return t

#this function checks which of the nodes is an extinct leaf or an ancestor of an extinct leaf
#and only leaves these nodes in the tree
def prune_nodes(t):
    for leaf in t.leaf_node_iter():    
        if hasattr(leaf, 'is_extinct'):
            leaf.extinct_ancestor = True
        else:
            leaf.extinct_ancestor = False
        
    for node in t.postorder_node_iter(): 
        if not hasattr(node, 'extinct_ancestor'):
            child_extinct = False
            for child in node.child_node_iter():
                if child.extinct_ancestor:
                    child_extinct =True
            node.extinct_ancestor = child_extinct
    labels = set([taxon.label for taxon in t.taxon_namespace
        if not t.find_node_for_taxon(taxon).extinct_ancestor])
    t1 = t.extract_tree_without_taxa_labels(labels=labels)
    return t1


#an example of a function to run birth-death simulations and generate output
def run_bd_simulations(num_trees, dimension):
    for i in range(num_trees):
        t = generate_birthdeath_tree(1,0.5,20)
        t= simulate_brownian(t, 1, dimension)
        d=dendropy.model.discrete.hky85_chars(kappa=3, mutation_rate=0.01, seq_len=1000,tree_model=t, retain_sequences_on_tree=False)
    
        for node in t.preorder_node_iter():
            node.annotations.add_bound_attribute("time")
            node.annotations.add_bound_attribute("X")
            if dimension==2:
                node.annotations.add_bound_attribute("Y")
    
        d.write(path="output1/fasta_output"+str(i)+".tre", schema="fasta")
        t.write(path="output2/phylogeny_output"+str(i)+".nex", schema="nexus", suppress_annotations=True)
        t.write(path="output3/newick_output"+str(i)+".tre", schema="newick", suppress_annotations=False)
        t.write(path="output4/nexus_output_"+str(i)+".tre", schema="nexus", suppress_internal_taxon_labels=True)    
        write_BEAST_xml(t, d, i, dimension)


In [6]:
import random

#function to generate coalescent trees (this is the ultrametric case)
def generate_ultrametric_coalescent_tree(num_tips):
    names = []
    lamb = 1
    
    #if there are N tips, there must be 2N-1 nodes
    for i in range(2*num_tips-1):
        names.append("T"+str(i))
    
    taxon_namespace = dendropy.TaxonNamespace(names)
    tree = dendropy.Tree(taxon_namespace=taxon_namespace)
    time_from_present = 0
    
    #current_nodes is a list of nodes that are currently not merged
    current_nodes = []
    for i in range(num_tips):
        node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(i)))
        current_nodes.append(node)
        node.age = 0 
    
    
    #if there are N leaves, there must be N-1 merges
    for merges in range(num_tips-1):
        #calculating time to the next coalescent
        time_to_coalescent=random.expovariate(lamb*len(current_nodes)*(len(current_nodes)-1)/2)
        
        time_from_present=time_from_present+time_to_coalescent
        
        #choosing 2 indices of nodes that will be merged  randomly
        merging_branches = random.sample(range(len(current_nodes)),2)
        node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(merges+num_tips)))
        
        #if it is the last merge, instead of creting a new node, we set the node of the merge to be the seed node
        if merges == num_tips-2:
            node=tree.seed_node
            node.taxon=taxon_namespace.get_taxon("T"+str(merges+num_tips))
        node.age = time_from_present
        current_nodes[merging_branches[0]].edge.length=time_from_present-current_nodes[merging_branches[0]].age
        current_nodes[merging_branches[1]].edge.length=time_from_present-current_nodes[merging_branches[1]].age
        node.set_child_nodes([current_nodes[merging_branches[0]], current_nodes[merging_branches[1]]])
        
        #deleting the nodes that have been merging from the list of nodes
        current_nodes.pop(max(merging_branches))
        current_nodes.pop(min(merging_branches))
        current_nodes.append(node)
        
    tree=calculate_times(tree)
    return tree


In [7]:
import random

#function to generate coalescent nonultrametric trees
def generate_coalescent_nonultrametric_tree():
    #a number num_tips_per_period is added every period_length of time units for num_periods times
    lamb=1
    period_length=1
    num_tips_per_period = 5
    num_periods = 3
    num_tips = num_tips_per_period*num_periods
    names = []
    
    #if there are N tips, there must be 2N-1 nodes
    for i in range(2*num_tips-1):
        names.append("T"+str(i))

    taxon_namespace = dendropy.TaxonNamespace(names)
    tree = dendropy.Tree(taxon_namespace=taxon_namespace)
    time_from_present = 0
    current_nodes = []
    
    #index variable simply keeps track of the number of nodes added just to choose the right name
    index = 0
    
    #simulating coalescent during every period
    for current_period in range(num_periods):
        time_from_present=current_period*period_length
        
        #adding new tips during this period
        for i in range(num_tips_per_period):
            node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(index)))
            current_nodes.append(node)
            index= index+1
            node.age = time_from_present
        
        
        current_num_tips = len(current_nodes)
        
        #iterating over the merges (if currently there are N unmerged nodes, there can be at most N-1 merges)
        for merges in range(current_num_tips-1):
            time_to_coalescent=random.expovariate(lamb*len(current_nodes)*(len(current_nodes)-1)/2)
            time_from_present=time_from_present+time_to_coalescent
            
            #if current time exceeds the end of current period, we stop the simulation of the period
            #and start simulating the next period
            if current_period < num_periods-1 and time_from_present > (current_period+1)*period_length:
                break
            else:
                merging_branches = random.sample(range(len(current_nodes)),2)
                
                if merges == current_num_tips-2 and current_period==num_periods-1:
                    node=tree.seed_node
                    node.taxon=taxon_namespace.get_taxon("T"+str(index))
                else:
                    node = dendropy.Node(taxon=taxon_namespace.get_taxon("T"+str(index)))
                index=index+1
                    
                node.age = time_from_present
                current_nodes[merging_branches[0]].edge.length=time_from_present-current_nodes[merging_branches[0]].age
                current_nodes[merging_branches[1]].edge.length=time_from_present-current_nodes[merging_branches[1]].age
                node.set_child_nodes([current_nodes[merging_branches[0]], current_nodes[merging_branches[1]]])
        
                current_nodes.pop(max(merging_branches))
                current_nodes.pop(min(merging_branches))
                current_nodes.append(node)
#     print(tree.as_ascii_plot(show_internal_node_labels=True, plot_metric='length'))
    tree=calculate_times(tree)
    return tree

t2=generate_coalescent_nonultrametric_tree()

In [8]:
num_trees = 5
dimension = 1
import beastxmlwriter
for i in range(num_trees):
    t = generate_coalescent_nonultrametric_tree()
    t= simulate_brownian(t, 1, dimension)
    d=dendropy.model.discrete.hky85_chars(kappa=3, mutation_rate=0.01, seq_len=1000,tree_model=t, retain_sequences_on_tree=False)
    beastxmlwriter.write_BEAST_xml(t, d, i, dimension)
    
    import os
    os.system('cmd /c java -jar beast.jar -overwrite -seed 1234 "output8\\beast'+str(i)+'.xml"')

TypeError: write_BEAST_xml() missing 1 required positional argument: 'log_every'

In [ ]:
def analyze_tree_list():
    treelist = dendropy.TreeList()
    treelist = dendropy.TreeList.get(path='beast_output\\beast0.trees.txt', schema="nexus", suppress_internal_node_taxa=False)
    for tree in treelist:
        print(tree.as_ascii_plot(plot_metric='length', show_internal_node_labels=True))
        break
analyze_tree_list()